In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq.chat_models import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from IPython.core.display import Markdown
from langchain_core.runnables import RunnableParallel
from langchain_core.messages import HumanMessage, AIMessage
import warnings

warnings.filterwarnings("ignore")
load_dotenv(find_dotenv())

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\xoxo3\OneDrive\Desktop\PYTHON\5_hr_rag_course\venv\Lib\site-packages\IPython\core\interactiveshell.py:3672: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [2]:
GROQ_API_KEY = os.getenv("GROQQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"]="advanced-rag"
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY

In [3]:
urls = [
    "https://www.webmd.com/a-to-z-guides/malaria",
    "https://www.webmd.com/diabetes/type-1-diabetes",
    "https://www.webmd.com/diabetes/type-2-diabetes",
    "https://www.webmd.com/migraines-headaches/migraines-headaches-migraines",
]

loader = WebBaseLoader(urls)

docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)
chunks = text_splitter.split_documents(docs)

embedding_function = HuggingFaceEmbeddings()

vecorstore = Chroma.from_documents(
    documents = chunks,
    embedding= embedding_function,
)

retriever = vecorstore.as_retriever(
    search_kwargs = {"k": 3}
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [5]:
retriever.get_relevant_documents("symptoms of malaria")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(metadata={'title': 'Malaria: Causes, Symptoms, and Treatment', 'language': 'en', 'source': 'https://www.webmd.com/a-to-z-guides/malaria', 'description': 'Malaria can be serious and sometimes life-threatening. Learn more about the disease and its new vaccine.'}, page_content="people have a higher risk of getting a serious case of malaria. These include:Children and babiesPeople who have HIV or AIDSPregnant peopleTravelers who live in areas without malariaMalaria SymptomsSymptoms usually start about 10-15 days after you're bitten by an infected mosquito. Symptoms can include:High feverChillsSweatingNausea or vomitingHeadacheDiarrheaBeing very tired (fatigue)Body achesYellow skin (jaundice)Kidney failureSeizureConfusionBloody stoolsConvulsionsDeathMalaria symptoms can"),
 Document(metadata={'description': 'Malaria can be serious and sometimes life-threatening. Learn more about the disease and its new vaccine.', 'language': 'en', 'title': 'Malaria: Causes, Symptoms, and Treatment

In [7]:
class VectorStore(BaseModel):
     (
        "A vectorstore contains information about symptoms, treatment"
        ", risk factors and other information about malaria, type 1 and"
        "type 2 diabetes and migraines"
    )
     
     query: str

class SearchEngine(BaseModel):
    """A search engine for searching other medical information on the web"""

    query: str

router_prompt_template = (
    "You are an expert in routing user queries to either a VectorStore, SearchEngine\n"
    "Use SearchEngine for all other medical queries that are not related to malaria, diabetes, or migraines.\n"
    "The VectorStore contains information on malaria, diabetes, and migraines.\n"
    'Note that if a query is not medically-related, you must output "not medically-related", don\'t try to use any tool.\n\n'
    "query: {query}"
)

llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0
)

prompt = ChatPromptTemplate.from_template(router_prompt_template)

question_router = prompt | llm.bind_tools(tools = [VectorStore, SearchEngine])

In [8]:
response = question_router.invoke("What are the symptoms of chest pain?")

"tool_calls" in response.additional_kwargs

True

In [9]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '144wnchq3', 'function': {'arguments': '{"query":"chest pain symptoms"}', 'name': 'SearchEngine'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 1087, 'total_tokens': 1132, 'completion_time': 0.167371658, 'prompt_time': 0.044401544, 'queue_time': 0.05973608699999999, 'total_time': 0.211773202}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--54b15e1a-f680-4f12-8d43-26ce994273e3-0', tool_calls=[{'name': 'SearchEngine', 'args': {'query': 'chest pain symptoms'}, 'id': '144wnchq3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1087, 'output_tokens': 45, 'total_tokens': 1132})

In [10]:
from langchain_core.pydantic_v1 import validator

class Grader(BaseModel):
    grade: Literal["relevant", "irrelevant"] = Field(
        ...,
        description= "The relevance score for the document.\n"
        "Set this to 'relevant' if the given context is relevant to the user's query, or 'irrlevant' if the document is not relevant.",
    )

    @validator("grade", pre = True)
    def validate_grade(cls, value):
        if value == "not relevant":
            return "irrelevant"
        return value

grader_system_prompt_template = """"You are a grader tasked with assessing the relevance of a given context to a query. 
    If the context is relevant to the query, score it as "relevant". Otherwise, give "irrelevant".
    Do not answer the actual answer, just provide the grade in JSON format with "grade" as the key, without any additional explanation."
    """

grader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", grader_system_prompt_template),
        ("human", "context: {context}\n\nquery: {query}"),
    ]
)

grader_chain = grader_prompt | llm.with_structured_output(Grader, method = "json_mode")

query = "What are the symptoms of malaria?"
context = retriever.get_relevant_documents(query)

response = grader_chain.invoke({
    "query": query,
    "context": context
})

In [11]:
print(response)

grade='relevant'


In [12]:
query = "Treatment of Ulcer"

context = retriever.get_relevant_documents(query)

response = grader_chain.invoke({
    "query": query,
    "context": context
})

print(response)

grade='irrelevant'


In [13]:
rag_template_str = (
    "You are a helpful assistant. Answer the query below based only on the provided context.\n\n"
    "context: {context}\n\n"
    "query: {query}"
)

rag_prompt = ChatPromptTemplate.from_template(rag_template_str)

rag_chain = rag_prompt | llm | StrOutputParser()

query = "what are the symptoms of malaria?"
context = retriever.get_relevant_documents(query)

response = rag_chain.invoke({
    "query": query,
    "context": context
})

Markdown(f"### Response:\n{response}")

### Response:
According to the provided context, the symptoms of malaria include:

1. High fever
2. Chills
3. Sweating
4. Nausea or vomiting
5. Headache
6. Diarrhea
7. Being very tired (fatigue)
8. Body aches
9. Yellow skin (jaundice)
10. Kidney failure
11. Seizure
12. Confusion
13. Bloody stools
14. Convulsions
15. Death
16. Tiredness and fatigue
17. Dark or bloody urine
18. Yellow eyes and skin (jaundice)
19. Abnormal bleeding

In [14]:
fallback_prompt = ChatPromptTemplate.from_template(
    (
       "You are a friendly medical assistant created by NHVAI.\n"
        "Do not respond to queries that are not related to health.\n"
        "If a query is not related to health, acknowledge your limitations.\n"
        "Provide concise responses to only medically-related queries.\n\n"
        "Current conversations:\n\n{chat_history}\n\n"
        "human: {query}" 
    )
)

fallback_chain = (
    {
        "chat_history": lambda x: "\n".join(
            [
                (
                    f"human: {msg.content}"
                    if isinstance(msg, HumanMessage)
                    else f"AI: {msg.content}"
                )
                for msg in x["chat_history"]
            ]
        ),
        "query": itemgetter("query"),
    }
    | fallback_prompt
    | llm
    | StrOutputParser()
)

fallback_chain.invoke(
    {
        "query": "Hello",
        "chat_history": [],
    }
)

"Hello! I'm happy to help with any health-related questions or concerns you may have. What's on your mind today?"

In [15]:
class HallucinationGrader(BaseModel):
    grade: Literal["yes", "no"] = Field(
        ...,
        description= "'yes' if the llm's reponse is hallucinated otherwise 'no'"
    )

hallucination_grader_system_prompt_template = (
    "You are a grader assessing whether a response from an llm is based on a given context.\n"
    "If the llm's response is not based on the given context give a score of 'yes' meaning it's a hallucination"
    "otherwise give 'no'\n"
    "Just give the grade in json with 'grade' as a key and a binary value of 'yes' or 'no' without additional explanation"
)

hallucination_grader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", hallucination_grader_system_prompt_template),
        ("human", "context: {context}\n\nllm's response: {response}"),
    ]
)

hallucination_grader_chain = (
    RunnableParallel(
        {
            "response": itemgetter("response"),
            "context": lambda x: "\n\n".join([c.page_content for c in x["context"]]),
        }
    )
    | hallucination_grader_prompt
    | llm.with_structured_output(HallucinationGrader, method = "json_mode")
)

query = "Symptoms of malaria"

context = retriever.get_relevant_documents(query)
response = """Based on the context provided, the symptoms of malaria include: Impaired consciousness, Convulsions, Difficulty breathing,
Serious tiredness and fatigue, Dark or bloody urine, Yellow eyes and skin (jaundice), Abnormal bleeding, High fever, Chills,
Sweating, Nausea or vomiting, Headache, Diarrhea"""

response = hallucination_grader_chain.invoke(
    {
        "response": response,
        "context": context
    }
)


In [16]:
response 

HallucinationGrader(grade='no')

In [17]:
class AnswerGrader(BaseModel):
    grade: Literal["yes", "no"] = Field(
        ...,
        description="'yes' if the provided answer is an actual answer to the query otherwise 'no'",
    )

answer_grader_system_prompt_template = (
    "You are a grader assessing whether a provided answer is in fact an answer to the given query.\n"
    "If the provided answer does not answer the query give a score of 'no' otherwise give 'yes'\n"
    "Just give the grade in json with 'grade' as a key and a binary value of 'yes' or 'no' without additional explanation"
)

answer_grader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", answer_grader_system_prompt_template),
        ("human", "query: {query}\n\nanswer: {response}"),
    ]
)

answer_grader_chain = answer_grader_prompt | llm.with_structured_output(
    AnswerGrader,
    method = "json_mode"
)

query = "Symptoms of malaria"

response = """Based on the context provided, the symptoms of malaria include: Impaired consciousness, Convulsions, Difficulty breathing,
Serious tiredness and fatigue, Dark or bloody urine, Yellow eyes and skin (jaundice), Abnormal bleeding, High fever, Chills,
Sweating, Nausea or vomiting, Headache, Diarrhea"""

response = answer_grader_chain.invoke(
    {
        "response": response,
        "query": query
    }
)

In [18]:
response 

AnswerGrader(grade='yes')

In [26]:
from typing import TypedDict
from langchain_core.documents import Document
from langgraph.prebuilt import ToolExecutor
from langchain_core.tools import Tool
from langchain_core.messages.base import BaseMessage

tavily_search = TavilySearchResults()
tool_executor = ToolExecutor(
    tools=[
        Tool(
            name="VectorStore",
            func=retriever.invoke,
            description="Useful to search the vector database",
        ),
        Tool(
            name="SearchEngine", func=tavily_search, description="Useful to search the web"
        ),
    ]
)


ImportError: cannot import name 'ToolExecutor' from 'langgraph.prebuilt' (c:\Users\xoxo3\OneDrive\Desktop\PYTHON\5_hr_rag_course\venv\Lib\site-packages\langgraph\prebuilt\__init__.py)